In [0]:
import dataiku
from langchain.chains.question_answering import load_qa_chain
from dataiku.langchain.dku_llm import DKUChatLLM
import json
KB_IDs = {
    "tech_docs": "zQ92IhQ9",
    "non_conformities": "WnKb6p17"
}

client = dataiku.api_client()
project = client.get_default_project()

In [0]:
# Listing available LLMs
llm_list = project.list_llms()

for llm in llm_list:
    print(f"- {llm.description} (id: {llm.id})")

In [0]:
# Fill with your LLM id
LLM_ID = "openai:OpenAI-FA:gpt-4o-mini"

In [0]:
# Preparing the Knowledge Bank, Vector store and LLM
KBs = {
    key: dataiku.KnowledgeBank(id=value, project_key=project.project_key)
    for key, value in KB_IDs.items()
}
vector_stores = {
    key: value.as_langchain_vectorstore()
    for key, value in KBs.items()
}

langchain_llm = DKUChatLLM(llm_id=LLM_ID, temperature=0)

In [0]:
# Create the question answering chain
chain = load_qa_chain(langchain_llm, chain_type="stuff")
user_message = "Oil"


In [0]:
role="000"
prompt = f"""
        Une non conformité de l'A220 doit être traitée selon le processus suivant :
            
            000 - rapport de non-conformité par le Quality Controler
            100 - analyse et recommandation / plan d'action par le Design Office
            200 - validation de l'analyse / plan d'action par le Design Manager
            300 - calcul de structure lié au plan d'action et recommandation / selon le Stress Office
            400 - du calcul / plan d'action amendé par le Stress Manager
            500 - plan d'action final validé par le Quality Manager
        
        You're supporting the role for {role} and rely on the knowledge from the A220 technical 
        doc and non conformity knowledge base (vector databases). You must provide an optimized expanded 
        prompt towards those vector databases to enable the best retrieval given the user input. 
        The expansion should only concern specificity around the user query and avoid retrieval of non specific
        vocabulary, as knowledge databses will contain any past non conformity. Avoid generic vocabulary like 
        'non-conformity', 'issue', 'specification', 'standard', 'operations', 'maintainance'. O expand 
        domain vocabulary.
        
        Format of the output: Please just provide the query in engish without any comment to be reused as is. 
        Optimal request should be between 20 and 50 words
        
        The user is the following:
        {user_message}
        
        
        Remember to only provide the requested query for the knowledge database without any comment.
    """
llm = project.get_llm(LLM_ID)
completion = llm.new_completion()
completion.with_message(prompt)
completion.settings["temperature"] = 0
resp = completion.execute()

print(resp.text)
query = resp.text

In [0]:
search_results = [result for key, value in vector_stores.items() for result in value.similarity_search(query)]
search_results = [ {
        "doc": s.metadata['doc'],
        "chunk_id": s.metadata['chunk_id'],
        "chunk": s.page_content
    }
    for s in search_results
]
print(search_results)

search_results = {
    key: value.similarity_search(query)
    for key, value in vector_stores.items()
}

for key in KB_IDs:
    for search_result in search_results[key]:
        print(f"# {search_result.doc} \n{search_result.page_content}\n")

In [0]:
        # 3rd step : give the best advice given the documents
        
prompt = """
    #Processus
    Une non conformité de l'A220 doit être traitée selon le processus suivant :

    000 - rapport de non-conformité par le Quality Controler
    100 - analyse et recommandation / plan d'action par le Design Office
    200 - validation de l'analyse / plan d'action par le Design Manager
    300 - calcul de structure lié au plan d'action et recommandation / selon le Stress Office
    400 - du calcul / plan d'action amendé par le Stress Manager
    500 - plan d'action final validé par le Quality Manager

    Vous supportez le role de l'étape {000} et devez rédiger de la facon la plus explicite en prenant
    les exemples fournis et la documentation technique.

    #Exemples et documentation technique:
    {json.dumps(search_results)}

    #La requête utilisateur est la suivante:
    {user_message}

    #Réponse
    ## Instruction globales du processus
    **Instructions du processus** :
    - **Analyse des causes** : Les causes doivent être réalistes et adaptées au contexte spécifique de l'A220, en tenant compte des impacts possibles.
    - **Causes internes et externes** : Différencier les causes internes (ex. : erreurs d'assemblage, calibrations incorrectes) et externes (ex. : défauts fournisseurs, intempéries).
    - **Orientation industrielle** : Prioriser les scénarios ayant un impact direct sur la navigabilité, la résistance (statique et fatigue), ou les coûts de production.

    **Orientation sur les gains industriels** :
    - **Réduction des coûts** : Prioriser les scénarios avec un impact financier élevé ou nécessitant des corrections coûteuses si elles ne sont pas détectées à temps.
    - **Efficacité temporelle** : Mettre en place des étapes d’analyse optimisées et des moyens de détection rapide pour réduire les délais de production.
    - **Pertinence industrielle** : Adopter une approche réaliste et contextuellement adaptée à l’industrie aéronautique, afin de garantir la navigabilité, la fiabilité et la conformité des produits.

    Les principes relatifs aux **non-conformités significatives** stipulent qu'une non-conformité qui peut affecter la navigabilité, la résistance (statique et fatigue), l’installation, le fonctionnement, ou tout autre domaine impactant la qualité et la sécurité doit être soigneusement évaluée et traitée. 

    Chaque **non-conformité significative** doit faire l'objet d'une demande de dérogation soumise à l'ingénierie pour une évaluation approfondie. Le processus de dérogation ne doit pas être utilisé pour des erreurs de conception ou des problèmes de configuration non anticipée. En outre, les **suffixes de dérogation** doivent être attribués pour définir les limitations permanentes ou temporaires sur les articles concernés.


    ## Instructions de réponse    
    Veuillez répondre pour l'étape {role}, en fournissant le meilleur 'label' et la meilleure 'description' possible selon les exemples, n'hésitant pas à illustrer selon les
    documentation technique le cas échéant. La description fournie doit être complètement rédigée.
    Si l'utilisateur a fourni un json avec un 'label' et une 'description' vous modifierez la description ou
    le titre selon les instruction de l'utilisateur, en maintenant un rôle de conseil vis à vis des exemples et
    de la documentation technique.

    
    
    ##Format de réponse
    Répondez en anglais sauf si l'utilisateur utilise une autre langue ou précise des instructions de langue.
    Format de réponse attendu en json sans autre mise en forme (pas de ```json). Vos commentaires sont fournis
    dans l'item 'comment':
    \\{label: ..., description: ..., comment: ...\\}
    - 'description' est en markdown. Dans tous les cas, le style reste technique et concis, 
    avec une approche plus télégraphique que rédigée de manière complexe (pas de phrases longues 
    ou compliquées). Faire comme dans les exemples, sans ajouter de termes de type "ce rapport précise",
    le rapport sera fourni dans un outil de ticketting, il faut rester concis et précis.
    - label : ne pas mentionner 'A220 Non-Conformity Report', juste le label de la non conformité, 
    comme dans les exemples
    - comment: fourni le cas échéant en markdwon pour l'interaction en mode canevas avec l'utilisateur {role}
"""

completion = llm.new_completion()
completion.with_message(prompt)
resp = completion.execute()

In [0]:
response_content = json.loads(resp.text)
deep_chat_response = {
    "text": response_content['comment'],
    "label": response_content['label'],
    "description": response_content['description'],
    "sources": search_results,
    "role": "ai"
}
print(deep_chat_response)